In [81]:
import pandas as pd

In [82]:
df_train = pd.read_csv('/Users/jamesliao2018/Logs/kaggle_data/competitive-data-science-predict-future-sales/sales_train.csv')

In [83]:
df_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [84]:
df_train['date2']=df_train['date'].apply(lambda x: x[3:])

In [85]:
df_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,date2
0,02.01.2013,0,59,22154,999.00,1.0,01.2013
1,03.01.2013,0,25,2552,899.00,1.0,01.2013
2,05.01.2013,0,25,2552,899.00,-1.0,01.2013
3,06.01.2013,0,25,2554,1709.05,1.0,01.2013
4,15.01.2013,0,25,2555,1099.00,1.0,01.2013


In [86]:
df_train.date2.max(),df_train.date2.min()

('12.2014', '01.2013')

In [87]:
df_export_all = df_train.groupby(['shop_id','item_id','date2']).item_cnt_day.sum().reset_index()

In [88]:
df_export_all = df_export_all.sort_values(['shop_id','item_id','date2'])

In [89]:
df_export_all_latest = df_export_all.drop_duplicates(['shop_id','item_id'],keep='last')

In [90]:
df_export_all_latest.head()

,shop_id,item_id,date2,item_cnt_day
0,0,30,02.2013,31.0
1,0,31,02.2013,11.0
3,0,32,02.2013,10.0
5,0,33,02.2013,3.0
7,0,35,02.2013,14.0


In [91]:
df_export_all_latest['date3']=df_export_all_latest['date2'].apply(lambda x: x[3:]+'.'+x[:2])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [92]:
df_export_all_latest.head()

,shop_id,item_id,date2,item_cnt_day,date3
0,0,30,02.2013,31.0,2013.02
1,0,31,02.2013,11.0,2013.02
3,0,32,02.2013,10.0,2013.02
5,0,33,02.2013,3.0,2013.02
7,0,35,02.2013,14.0,2013.02


In [93]:
df_export_all_latest.date3.min(),df_export_all_latest.date3.max()

('2013.01', '2015.10')

In [94]:
df_export_all_latest['sales2'] = df_export_all_latest.apply(lambda x: \
                                                            x['item_cnt_day'] if x['date3']<='2015.09' else x['item_cnt_day'],\
                                                           axis=1)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [95]:
df_export_final = df_export_all_latest[['shop_id','item_id','sales2']]

In [96]:
df_export_final.shape

(424124, 3)

In [97]:
df_train_sku_sale=df_train.groupby('item_id').item_cnt_day.sum().reset_index()

In [98]:
df_train_sku_sale.sort_values('item_cnt_day',ascending=False).head()

,item_id,item_cnt_day
20602,20949,187642.0
2749,2808,17245.0
3654,3732,16642.0
17418,17717,15830.0
5717,5822,14515.0


In [99]:
df_train_sku_sale.sort_values('item_cnt_day').head()

,item_id,item_cnt_day
1552,1590,-11.0
13223,13474,-1.0
11664,11871,-1.0
17760,18062,-1.0
13226,13477,0.0


In [100]:
df_test = pd.read_csv('/Users/jamesliao2018/Logs/kaggle_data/competitive-data-science-predict-future-sales/test.csv')

In [101]:
df_test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [102]:
df_test.shape

(214200, 3)

In [103]:
df_submit = df_test.merge(df_export_final,on=['shop_id','item_id'],how='left')

In [104]:
df_submit.head()

,ID,shop_id,item_id,sales2
0,0,5,5037,2.0
1,1,5,5320,NaN
2,2,5,5233,1.0
3,3,5,5232,1.0
4,4,5,5268,NaN


In [105]:
df_submit[df_submit.sales2.isnull()].shape

(102796, 4)

In [106]:
df_submit.loc[df_submit.sales2.isnull(),'sales2']=0

In [107]:
df_submit.head()

,ID,shop_id,item_id,sales2
0,0,5,5037,2.0
1,1,5,5320,0.0
2,2,5,5233,1.0
3,3,5,5232,1.0
4,4,5,5268,0.0


In [108]:
df_submit=df_submit.rename(columns={'sales2':'item_cnt_month'})

In [109]:
df_submit.shape

(214200, 4)

In [110]:
df_submit[['ID','item_cnt_month']].to_csv('submission_baseline_0605.csv',index=False)

In [12]:
df_submission_sample = pd.read_csv('/Users/jamesliao2018/Logs/kaggle_data/competitive-data-science-predict-future-sales/sample_submission.csv')

In [13]:
df_submission_sample.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [52]:
df_submission_sample.shape

(214200, 2)

In [14]:
df_cat = pd.read_csv('/Users/jamesliao2018/Logs/kaggle_data/competitive-data-science-predict-future-sales/item_categories.csv')

In [23]:
df_cat.shape

(84, 2)

In [15]:
df_cat.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [18]:
df_cat.tail()

,item_category_name,item_category_id
79,Служебные,79
80,Служебные - Билеты,80
81,Чистые носители (шпиль),81
82,Чистые носители (штучные),82
83,Элементы питания,83


In [16]:
df_item = pd.read_csv('/Users/jamesliao2018/Logs/kaggle_data/competitive-data-science-predict-future-sales/items.csv')

In [17]:
df_item.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [24]:
df_item.shape

(22170, 3)

In [46]:
df_item[df_item['item_id']==5320]

,item_name,item_id,item_category_id
5320,ONE DIRECTION Made In The A.M.,5320,55


In [25]:
df_item_group = df_item.groupby('item_category_id').item_name.nunique().reset_index()

In [26]:
df_item_group.head()

,item_category_id,item_name
0,0,4
1,1,2
2,2,75
3,3,34
4,4,15
